In [241]:
# import pandas
import pandas as pd

In [242]:
# อ่านไฟล์ Dataset

# ระบุ path
path = "Data.txt"

# อ่านมาเป็น df
df = pd.read_csv(path, header=None)

# ตรวจดูตัวอย่าง 5 แถวแรก
print(df.head())

# ดูขนาดตาราง (แถว, คอลัมน์)
print(df.shape)


       0   1       2                 3                   4      5
0  White  39    Male         State-gov       Never-married  <=50K
1  White  50    Male  Self-emp-not-inc  Married-civ-spouse  <=50K
2  White  38    Male           Private            Divorced  <=50K
3  Black  53    Male           Private  Married-civ-spouse  <=50K
4  Black  28  Female           Private  Married-civ-spouse  <=50K
(30725, 6)


In [243]:
# กำหนด header
cols = ["Race", "Age", "Sex", "Workclass", "Marital-status", "Income/year"]

# กำหนด header ให้ DataFrame
df.columns = cols

# ตรวจสอบ
print(df.head())

    Race  Age     Sex         Workclass      Marital-status Income/year
0  White   39    Male         State-gov       Never-married       <=50K
1  White   50    Male  Self-emp-not-inc  Married-civ-spouse       <=50K
2  White   38    Male           Private            Divorced       <=50K
3  Black   53    Male           Private  Married-civ-spouse       <=50K
4  Black   28  Female           Private  Married-civ-spouse       <=50K


In [244]:
# ตัด attribute "Income/year" ออก 
df.drop(columns=["Income/year"], inplace=True)

# ตรวจสอบ
print(df.head())

    Race  Age     Sex         Workclass      Marital-status
0  White   39    Male         State-gov       Never-married
1  White   50    Male  Self-emp-not-inc  Married-civ-spouse
2  White   38    Male           Private            Divorced
3  Black   53    Male           Private  Married-civ-spouse
4  Black   28  Female           Private  Married-civ-spouse


In [245]:
# กำหนดประเภท Attribute
SENSITIVE_ATTRS = ["Workclass"] 
QI_ATTRS = [c for c in df.columns if c not in SENSITIVE_ATTRS]

In [246]:
# กำหนด VGH

# AGE
def vgh_age(a, level: int):
    # level: 0=exact, 1=1-25/26-50/51-75/76-100, 2=1-50/51-100, 3='*'
    if level == 3: return "*"
    if level == 2: return "1-50" if a in {"1-25", "26-50"} else "51-100"
    if level == 1:
        a = int(a)
        if   1 <= a <= 25:  return "1-25"
        if  26 <= a <= 50:  return "26-50"
        if  51 <= a <= 75:  return "51-75"
        return "76-100"
    if level == 0: return str(a)

# SEX
def vgh_sex(s, level: int):
    # level: 0=Female/Male, 1='*'
    if level == 1: return "*"
    if level == 0: return str(s)

# RACE
def vgh_race(r, level: int):
    # level: 0=5 หมวด, 1=รวม {Amer-Indian-Eskimo,Other}, 2=White/Non-White, 3='*'
    r = str(r)
    if level == 3: return "*"
    if level == 2: return "White" if r == "White" else "Non-White"
    if level == 1: return "Amer-Indian-Eskimo+Other" if r in {"Amer-Indian-Eskimo", "Other"} else r
    if level == 0: return str(r)

# MARITAL
def vgh_marital(m, level: int):
    # level: 0=7 ค่าเดิม, 1=Married/Previously-married/Never-married, 2=Married/Not-married, 3='*'
    m = str(m)
    if level == 3: return "*"
    if level == 2: return "Married" if m in {"Married"} else "Not-married"
    if level == 1:
        if m in {"Married-civ-spouse", "Married-AF-spouse"}: return "Married"
        if m in {"Divorced", "Separated", "Widowed", "Married-spouse-absent"}: return "Previously-married"
        return "Never-married"
    if level == 0: return str(m)

# จำกัดระดับสูงสุดของแต่ละ QI Attribute (ตาม VGH/DGH ที่กำหนด)
MAX_LVL = {
    "Age": 3,              # 0..3
    "Race": 3,             # 0..3
    "Marital-status": 3,   # 0..3
    "Sex": 1,              # 0..1
}

In [247]:
# กำหนด K
TARGET_K = 4

In [248]:
# นำข้อมูลไปใส่ตัวแปรสำหรับดำเนินการ
df_process = df.copy()

# กำหนดระดับ Domain ให้แต่ละุ record เริ่มต้นเป็น 0
df_process["A_lvl"] = 0
df_process["R_lvl"] = 0
df_process["M_lvl"] = 0
df_process["S_lvl"] = 0

# ตัวแปรเก็บข้อมูล record ที่ผ่าน k แล้ว
df_success = pd.DataFrame(columns=df_process.columns)

In [249]:
# Function ตรวจค่า K และจับแยก
def split_by_k(df_proc, df_succ, k):
    # จับ EC
    ec = df_proc.groupby(QI_ATTRS, dropna=False).size().rename("ec_size").reset_index()
    tmp = df_proc.merge(ec, on=QI_ATTRS, how="left")
    # Mark Record ที่ผ่านค่า k
    mask_pass = tmp["ec_size"] >= k
    # แยก Record ที่ผ่านค่า k ไปไว้ใน df_succ
    df_succ = pd.concat([df_succ, tmp.loc[mask_pass].drop(columns=["ec_size"])], ignore_index=True)
    # ที่เหลือใส่ใน df_proc
    df_proc = tmp.loc[~mask_pass].drop(columns=["ec_size"]).copy()
    return df_proc, df_succ

In [250]:
# ลำดับรอบ-โรบิน
ATTR_ORDER = ["Age", "Race", "Marital-status", "Sex"]
start_idx = 0  # เริ่มจาก Age (index 0)


# map ชื่อคอลัมน์ -> ฟังก์ชัน VGH ที่นิยามไว้
APPLY_VGH = {
    "Age": vgh_age,
    "Race": vgh_race,
    "Marital-status": vgh_marital,
    "Sex": vgh_sex,
}

# คอลัมน์เก็บระดับของแต่ละ QI (คุณมี A_lvl/R_lvl/M_lvl/S_lvl แล้ว)
LVL_COL = {"Age": "A_lvl", "Race": "R_lvl", "Marital-status": "M_lvl", "Sex": "S_lvl"}

max_iters = 200
for it in range(max_iters):

    # จับ EC → ตรวจ k → แยกผ่าน/ไม่ผ่าน
    df_process, df_success = split_by_k(df_process, df_success, TARGET_K)
    print(f"[รอบที่่ {it+1}] ผ่าน {len(df_success)} - ไม่ผ่าน {len(df_process)}")

    # ถ้า df_process หมดแล้วให้หยุด
    if len(df_process) == 0:
        print("------------------------------------------------------------------------------")
        print(f"เสร็จสิ้น!! ทุก Record ถูกทำให้ผ่านค่า K")
        # เครียร์ Attribute ที่บอก Domain ออก
        df_success.drop(columns=["A_lvl"], inplace=True)
        df_success.drop(columns=["R_lvl"], inplace=True)
        df_success.drop(columns=["M_lvl"], inplace=True)
        df_success.drop(columns=["S_lvl"], inplace=True)
        # แสดงตัวอย่างข้อมูล
        print(df_success)
        df_success.to_csv("result_success.csv", index=False, encoding="utf-8")
        break

    # ถ้า record ใน df_process เหลือ < k ให้ลบ record ทิ้งแล้วหยุด
    if len(df_process) < TARGET_K:
        print(f"!!!! Record ที่ยังไม่ผ่านค่า K เหลือ {len(df_process)} แถว ซึ่งน้อยกว่าค่า K = {TARGET_K} -> ลบ")
        # ลบ Record ที่เหลือทิ้ง
        df_process = df_process.iloc[0:0].copy()
        print("------------------------------------------------------------------------------")
        print(f"เสร็จสิ้น!! แบบมีการลบบาง Record")
        # เครียร์ Attribute ที่บอก Domain ออก
        df_success.drop(columns=["A_lvl"], inplace=True)
        df_success.drop(columns=["R_lvl"], inplace=True)
        df_success.drop(columns=["M_lvl"], inplace=True)
        df_success.drop(columns=["S_lvl"], inplace=True)
        # แสดงตัวอย่างข้อมูล
        print(df_success)
        df_success.to_csv("result_success.csv", index=False, encoding="utf-8")
        break

    # 4) ยกระดับ df_process ขึ้นทีละ 1 ระดับ แบบรอบ-โรบิน
    bumped = False
    for step in range(len(ATTR_ORDER)):
        attr = ATTR_ORDER[(start_idx + step) % len(ATTR_ORDER)]
        lvlc = LVL_COL[attr]
        # หา Record ที่ attr level ยังไม่ max
        can = df_process[lvlc] < MAX_LVL[attr]
        if not can.any():
            continue

        # gen ค่าใหม่และเขียนทับลงไป
        fn = APPLY_VGH[attr]
        df_process.loc[can, attr] = df_process.loc[can].apply(
            lambda r: fn(r[attr], r[lvlc] + 1), axis=1
        )
        # อัปเดต domain level ของ Attribute นั้น +1
        df_process.loc[can, lvlc] = df_process.loc[can, lvlc] + 1

        bumped = True
        # กำหนด Attribute ตัวถัดไปที่จะ gen ในรอบถัดไป
        start_idx = (start_idx + step + 1) % len(ATTR_ORDER)
        break

    # ถ้า gen ไม่ได้แล้ว (ทุก Attribute ชนเพดาน) ให้ลบทิ้งส่วนที่เหลือ
    if not bumped:
        print(f"!!!! เพิ่มระดับต่อไม่ได้แล้ว (ชนเพดานทุกคอลัมน์) -> ลบ {len(df_process)} แถวที่ยังไม่ผ่าน")
        # ลบ Record ที่เหลือทิ้ง
        df_process = df_process.iloc[0:0].copy()
        print("------------------------------------------------------------------------------")
        print(f"เสร็จสิ้น!! แบบมีการลบบาง Record")
        # เครียร์ Attribute ที่บอก Domain ออก
        df_success.drop(columns=["A_lvl"], inplace=True)
        df_success.drop(columns=["R_lvl"], inplace=True)
        df_success.drop(columns=["M_lvl"], inplace=True)
        df_success.drop(columns=["S_lvl"], inplace=True)
        # แสดงตัวอย่างข้อมูล
        print(df_success)
        df_success.to_csv("result_success.csv", index=False, encoding="utf-8")
        break

[รอบที่่ 1] ผ่าน 29288 - ไม่ผ่าน 1437
[รอบที่่ 2] ผ่าน 30607 - ไม่ผ่าน 118
[รอบที่่ 3] ผ่าน 30620 - ไม่ผ่าน 105
[รอบที่่ 4] ผ่าน 30656 - ไม่ผ่าน 69
[รอบที่่ 5] ผ่าน 30679 - ไม่ผ่าน 46
[รอบที่่ 6] ผ่าน 30689 - ไม่ผ่าน 36
[รอบที่่ 7] ผ่าน 30705 - ไม่ผ่าน 20
[รอบที่่ 8] ผ่าน 30711 - ไม่ผ่าน 14
[รอบที่่ 9] ผ่าน 30723 - ไม่ผ่าน 2
!!!! Record ที่ยังไม่ผ่านค่า K เหลือ 2 แถว ซึ่งน้อยกว่าค่า K = 4 -> ลบ
------------------------------------------------------------------------------
เสร็จสิ้น!! แบบมีการลบบาง Record
        Race Age     Sex         Workclass      Marital-status
0      White  39    Male         State-gov       Never-married
1      White  50    Male  Self-emp-not-inc  Married-civ-spouse
2      White  38    Male           Private            Divorced
3      Black  53    Male           Private  Married-civ-spouse
4      Black  28  Female           Private  Married-civ-spouse
...      ...  ..     ...               ...                 ...
30718  White   *       *           Private       

C:\Users\vrzo2\AppData\Local\Temp\ipykernel_26028\1138017566.py:67: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '['26-50' '26-50' '51-75' ... '51-75' '26-50' '26-50']' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_process.loc[can, attr] = df_process.loc[can].apply(


In [ ]:
ec = df_success.groupby(QI_ATTRS, dropna=False).size().rename("ec_size").reset_index().sort_values("ec_size", ascending=True)  # เรียงจากมากไปน้อย
print(ec)
min = ec.min

                         Race     Age     Sex      Marital-status  ec_size
572                     White      37    Male  Married-civ-spouse      409
580                     White      38    Male  Married-civ-spouse      397
610                     White      41    Male  Married-civ-spouse      375
563                     White      36    Male  Married-civ-spouse      374
543                     White      34    Male  Married-civ-spouse      370
..                        ...     ...     ...                 ...      ...
30   Amer-Indian-Eskimo+Other   26-50  Female             Widowed        4
32   Amer-Indian-Eskimo+Other  51-100       *  Previously-married        4
35   Amer-Indian-Eskimo+Other   51-75    Male  Previously-married        4
874                     White    1-25  Female   Married-AF-spouse        4
877                     White    1-25    Male  Married-civ-spouse        4

[903 rows x 5 columns]
